# 1. Introduktion
Vi vil ved hjælp af diverse pakker udviklet til python kigge på vores datasæt, hvor vi vil arbejde med demografisk filtrering, se på fordele og ulemper med demografisk filtrering dernæst vil vi arbejde med indhold baseret filtrering.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
from ast import literal_eval
import nltk
from nltk.corpus import stopwords
from nltk.util import bigrams
import math
from sklearn.preprocessing import MinMaxScaler
import pickle

In [ ]:
df_credits = pd.read_csv("data/tmdb_5000_credits.csv")
df_movies = pd.read_csv("data/tmdb_5000_movies.csv")

Efter at vi har læst vores data filer, vil vi gerne se nærmerer på dem, for at skabe en generel forståelse for datasættets indhold.

In [ ]:
df_credits.head()

In [ ]:
df_movies.head()

Det ses at begge datasæt indeholder en variabel kaldet "id" og "movie_id". Dette er en reference variabel der er ens for begge datasæt, det betyder, at en vilkårlig film med eksempelvis titel "Avatar" har "id" = 19995 og det samme er gældende for "movie_id" med samme titel, har også værdien 19995.

Vi vil gerne samle vores data, så vi tilføjer "crew" og "cast" til vores film datasæt og dette kan vi gøre på id'et, da de er ens for begge datasæt. Inden vi gør det, fjerner vi variablen "title" fra vores "credits" datasæt, da titlen er redundant og dermed allerede eksistere i vores film datasæt.

In [ ]:
df_credits.drop('title', axis=1, inplace=True)
df_credits.columns = ['id', 'cast', 'crew']
main_df = df_movies.merge(df_credits, on="id")
main_df.head()

In [ ]:
main_df.columns

In [ ]:
main_df.describe()

In [ ]:
main_df.shape

In [ ]:
main_df.size

In [ ]:
df_info=pd.DataFrame(main_df.dtypes).T.rename(index={0:'column type'})
df_info=df_info.append(pd.DataFrame(main_df.isnull().sum()).T.rename(index={0:'null values'}))
df_info=df_info.append(pd.DataFrame(main_df.isnull().sum()/main_df.shape[0]*100).T.rename(index={0:'null values (%)'}))
df_info

In [ ]:
def nan_in_percentages():
    non_nan = []
    [non_nan.append(val) for val in df_info.loc['null values'] if val == 0]
    non_nan = [1 - len(non_nan) / len(df_info.loc['null values (%)']), len(non_nan) / len(df_info.loc['null values'])]
    return non_nan

In [ ]:
# make figure and assign axis objects
fig = plt.figure(figsize=(9, 5.0625)) #5.0625
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
fig.subplots_adjust(wspace=0)

# pie chart parameters
ratios = nan_in_percentages()
labels = ['NaN values', 'Non NaN values']
explode = [0.1, 0]
# rotate so that first wedge is split by the x-axis
angle = -180 * ratios[0]
ax1.pie(ratios, autopct='%1.1f%%', startangle=angle,
        labels=labels, explode=explode)

# bar chart parameters

xpos = 0
bottom = 0
ratios = sorted([val/sum(df_info.loc['null values'].values) for val in df_info.loc['null values'] if val != 0], reverse=False)
width = .2
colors = [[.1, .3, .1], [.1, .3, .9], [.1, .3, .7], [.1, .3, .3], [.1, .3, .5]]

for j in range(len(ratios)):
    height = ratios[j]
    ax2.bar(xpos, height, width, bottom=bottom, color=colors[j])
    ypos = bottom + ax2.patches[j].get_height() / 2
    bottom += height
    ax2.text(xpos, ypos, "%d%%" % (ax2.patches[j].get_height() * 100),
             ha='center')

ax2.set_title('Has NaN values')
ax2.legend(([df_info.loc['null values'].sort_values(ascending=True).index[col] for col, val in enumerate(df_info.loc['null values'].sort_values(ascending=True)) if val != 0]))
ax2.axis('off')
ax2.set_xlim(-2.5 * width, 2.5 * width)

# use ConnectionPatch to draw lines between the two plots
# get the wedge data
theta1, theta2 = ax1.patches[0].theta1, ax1.patches[0].theta2
center, r = ax1.patches[0].center, ax1.patches[0].r
bar_height = sum([item.get_height() for item in ax2.patches])

# draw top connecting line
x = r * np.cos(np.pi / 180 * theta2) + center[0]
y = np.sin(np.pi / 180 * theta2) + center[1]
con = ConnectionPatch(xyA=(- width / 2, bar_height), xyB=(x, y),
                      coordsA="data", coordsB="data", axesA=ax2, axesB=ax1)
con.set_color([0, 0, 0])
con.set_linewidth(4)
ax2.add_artist(con)

# draw bottom connecting line
x = r * np.cos(np.pi / 180 * theta1) + center[0]
y = np.sin(np.pi / 180 * theta1) + center[1]
con = ConnectionPatch(xyA=(- width / 2, 0), xyB=(x, y), coordsA="data",
                      coordsB="data", axesA=ax2, axesB=ax1)
con.set_color([0, 0, 0])
ax2.add_artist(con)
con.set_linewidth(4)

#plt.savefig('testpiepig_pandas_900dpi.svg', dpi=900)
plt.show()

Det ses på grafen ovenfor, at det 77,3% af alle variabler i vores datasæt indeholder data, mens 22,7% ikke gør, men ud af de 22,7% er 78% af dem data der mangler på variablen "homepage". <br>
"homepage" variablen er sammentidigt en variabel vi ikke kan bruge til noget i forhold til det initierende problem, som er: Der ønskes film anbefalinger baseret på det givne datasæt. <br>
21% er variablen "tagline" der indeholder noget beskrivende om filmen handling - Denne variabel vil være oplagt at lave TF-IDF på, men da den indeholder så mange NaN værdier på man stærkt overveje, om variablen stadig bør anvendes. <br>
"realease_date", "runtime", "overview" udgør hver især mindre end 1% og derfor er der tale om få NaN værdier i disse variabler.

# Top 10 film

### Top 10 film baseret på den gennemsnitlige bedømmelse
Nu når alt data er samlet ét datasæt, er vi nu interesseret, at give nogle film anbefalinger ud fra det data vi er givet.
Dette kan lade sig gøre ved at se på variablen vote_average.

In [ ]:
main_df[["title", "release_date", "vote_average", "vote_count"]].sort_values("vote_average", ascending=False).head(10)

Det ses på tabellen ovenfor de 10 bedst bedømte film.<br>
Dette er ikke særlig given, da vi kan se nogle af de bedst rated film kun har 1 til 2 stemmer.

### Top 10 film baseret på det vægtede gennemsnit

Derfor vil vi lave en ny rating variabel der giver filmene en ny rating der også betragter antal stemmer den enkelte film har fået. Dette kan gøres ved hjælp af noget kaldt "Weighted Rating", der kan skrives som:<br>
Weighted Rating (WR)=(v/(v+m))R+(m/(v+m))C<br>
hvor:<br>
R = gennemsnits rating for filmen = "vote_average".<br>
v = antallet af stemmer for filmen = "vote_count".<br>
m = minimum antal stemmer krævet, for at være en "gyldig" film.<br>
C = gennemsnits rating for alle film.<br>

In [ ]:
m = main_df['vote_count'].quantile(0.9)
C = main_df["vote_average"].mean()

main_df["WR"] = (main_df["vote_count"]/(main_df["vote_count"] + m)) * main_df["vote_average"] + (m/(main_df["vote_count"] + m)) * C

##### Det nye resultat
Der sorteres nu på de 10 bedste film baseret på vores nye variabel kaldet "WR" og det ses nu, at de film anbefalet baseret på WR er langt fra de 10 film vi fik anbefalet tidligere, dette skyldes at der vægtes på antallet af stemmer en film har modtaget.

In [ ]:
main_df[['title', "release_date", 'vote_average', 'vote_count', 'WR']].sort_values("WR", ascending=False).head(10)

# Populære film

Vi kan nu give en komfortabel anbefaling af film baseret på deres bedømmelse, men nogle af de film er af ældre dato, hvilket leder os frem til spørgsmåle - hvad der er populært lige nu og her?

In [ ]:
popular_movies = main_df.sort_values("popularity", ascending=False)

plt.figure(figsize=(14,8))

plt.bar(popular_movies['title'].head(10),popular_movies['popularity'].head(10), align='center', color='#211a52')
plt.xticks(rotation=90)
plt.xlabel("Popularity")
plt.title("Top 10 Popular Movies")
plt.show()

## Skaleret Score

Vi har nu de film der trender og de film, der har en høj rating. <br>
Hvis disse to variabler ønskes kombineret og vægtes ligeligt. <br>
Det vil resulterer i at WR og popularity vil vægte 50% hver især.

In [ ]:
ss = MinMaxScaler().fit_transform(main_df[['popularity','WR']])

In [ ]:
def scaled_score(x):
    return [(0.5 * ss[i][0]) + (0.5 * ss[i][-1]) for i in range(x)]

In [ ]:
main_df['scaled_score'] = scaled_score(len(main_df.index))

In [ ]:
plt.figure(figsize=(14,8))
plt.bar(main_df.sort_values("scaled_score", ascending=False)['title'].head(10), 
         main_df.sort_values("scaled_score", ascending=False)['scaled_score'].head(10), 
         align='center', color='#211a52')
plt.xticks(rotation=90)
plt.xlabel("Rating & Popularity Combined (Hybrid Score)")
plt.title("Top 10 Movies with Scaled Scores")
plt.show()

### Data cleaning
Selvom vi har nogle løsninger i form af anbefalinger er løsningerne meget overfaldiske og generelle, da de ikke tager højde for den enkelte brugers preferencer. <br>
Dette vil vi tage højde for nu, ved at indtaste eksempelvis sin yndlingsfilm og ud fra den film finde film der minder om den indtastede film.<br>
Da mange af variablerne indeholder tekst værdier, er det nødvendigt at lave tekst om til noget vi kan regne på eksempelvis med tal værdier. Dette kan lade sig gøre ved hjælp af TD-IDF. <br>
Der eksisterer dog nogle variabler i datasættet der har værdier vi gerne vil rense, for unødvendige værdier og omskrive til nogle håndgribelige værdier, så vi kan udfører vores tekst analyse.

In [ ]:
cleaned_df = main_df

In [ ]:
# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
# Get all the "names" from genre, cast, crew, keywords, productions_companies and spoken_languages.
# If there are no "names" return an empty list
def get_list(x):
    return [i['name'] for i in x] if isinstance(x, list) else []

In [ ]:
features = ['cast', 'crew', 'keywords', 'genres', 'production_companies', 'production_countries', 'spoken_languages']
for feature in features:
    cleaned_df[feature] = cleaned_df[feature].apply(literal_eval)

# Define new director, cast, genres and keywords features that are in a suitable form.
cleaned_df['director'] = cleaned_df['crew'].apply(get_director)

for feature in features:
    cleaned_df[feature] = cleaned_df[feature].apply(get_list)

#### Sådan ser de nye variabler ud efter noget rensning

In [ ]:
cleaned_df[features].head()

# Grafisk oversigt
Her gives et generelt overblik over de forskellige værdier i datasættet.

In [ ]:
plt.figure(figsize=(12,10))
plt.bar(main_df.sort_values("budget", ascending=False)['title'].head(10), 
        main_df.sort_values("budget", ascending=False)['budget'].head(10), 
        align='center', color='#211a52')
plt.xlabel("Budget in $")
plt.ticklabel_format(axis='y', style='plain')
plt.xticks(rotation=90)
plt.title("Top 10 Movies with Highest Budget")
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
plt.bar(main_df.sort_values("revenue", ascending=False)['title'].head(10), 
         main_df.sort_values("revenue", ascending=False)['revenue'].head(10), 
         align='center', color='#211a52')
#plt.gca().invert_yaxis()
plt.xlabel("Revenue in $")
plt.xticks(rotation=90)
plt.ticklabel_format(axis='y', style='plain')
#plt.xaxis.get_offset_text().set_visible(False)
plt.title("Top 10 Movies with Highest Revenue")
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
plt.bar(main_df.sort_values("runtime", ascending=False)['title'].head(10), 
         main_df.sort_values("runtime", ascending=False)['runtime'].head(10), 
         align='center', color='#211a52')
#plt.gca().invert_yaxis()
plt.xlabel("Watch time in minutes")
plt.xticks(rotation=90)
plt.ticklabel_format(axis='y', style='plain')
#plt.xaxis.get_offset_text().set_visible(False)
plt.title("Top 10 Longest to Watch Movies")
plt.show()

In [ ]:
def vertical_bar_plot(data, title, len_return_val):
    plt.subplots(figsize=(12,10))
    
    data_list = []
    for i in data:
        if isinstance(i, str):
            if i != '':
                data_list.append(i)
        else:
            data_list.extend(i)

    ax = pd.Series(data_list).value_counts()[:len_return_val].sort_values(ascending=False).plot.bar(width=0.9,color='#211a52')
    
    #for i, v in enumerate(pd.Series(data_list).value_counts()[:len_return_val].sort_values(ascending=False).values): 
        #ax.text(.8, i, v, fontsize=12, color='white', weight='bold')
    
    plt.title(title)
    plt.show()

In [ ]:
vertical_bar_plot(cleaned_df['genres'], 'Top Genres', 10)

In [ ]:
vertical_bar_plot(cleaned_df['cast'], 'Actors with most appearance', 10)

In [ ]:
vertical_bar_plot(cleaned_df['production_companies'], 'Production Companies with most associations', 10)

In [ ]:
vertical_bar_plot(cleaned_df['director'].fillna(''), 'Directors with most movies', 10)

In [ ]:
vertical_bar_plot(cleaned_df['keywords'], 'Most used keywords', 10)

In [ ]:
vertical_bar_plot(cleaned_df['production_countries'], 'Production Countries with most movies', 5)

In [ ]:
vertical_bar_plot(cleaned_df['spoken_languages'], 'Most spoken language', 5)

In [ ]:
year_list = []
for date in cleaned_df['release_date'].fillna('').str.split('-'):
    year_list.append(date[0])
vertical_bar_plot(year_list, 'Year with most released movies', 5)

In [ ]:
cleaned_df.drop(columns=['homepage', 'original_language', 'tagline', 'original_title', 'runtime', 'spoken_languages', 'vote_average', 'vote_count', 'status', 'production_countries', 'crew', 'budget', 'revenue', 'release_date', 'genres', 'production_companies'], inplace=True)

In [ ]:
cleaned_df.columns

# TF-IDF og Content Based Filtering
Det er nu tid til at påbegynde den del, der anbefaler eksempelvis 10 film baseret på sin yndlingsfilm eller en hvilken som helst anden given film.

In [ ]:
# It is required to install stopwords, before running this code. This is done by:
# >conda install -c anaconda nltk
# >python
# >>>import nltk
# >>>nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.update(',', '―', ';', '!', '?', '.', '(', ')', '$', '#', '+', ':', '...', ' ', '', "he's","who's", "it's", 'hes', 'shes', 'whos', '--', '–')
print(stop_words)

In [ ]:
#Replace NaN with an empty string
#cleaned_df['overview'] = cleaned_df['overview'].fillna('') # Or drop NA?
#cleaned_df = cleaned_df[cleaned_df['overview'].notna()]
cleaned_df.drop(cleaned_df.loc[cleaned_df['overview'] == ' '].index, inplace=True)
cleaned_df.dropna(subset=['overview'], inplace=True)
cleaned_df.reset_index(inplace=True)

#### overview_cleaning funkltionen
Denne funktion rengører vores film beskrivelse ved hjælp af stopwords og en ekstra filtrering i form af special tegn og andet.
<br>
<br>
OBS - Det er vigtigt at påpege, at tf-idf ikke kan genkende synonymer eller forstår stavefejl eller sætningen i sig selv, men kigger kun på ordet eller tekst strengen i sig selv. Dette vil resulterer i at eksempelvis at peter og peters ikke er det samme, ligesom at peters og peter's heller ikke har samme betydning.
Da vores liste af unikke ord efter filtrering af stopwords og alt andet indeholder stadig 24067 ord og der er med en statistisk sikkerhed ord der burde være "ens" men ikke er, da ordet har en apostrof eller andet der differentierer den fra det originale ord.

In [ ]:
def overview_cleaning(x):
    x = x.lower().strip('[]').replace('"','').replace('!','').replace('?','').replace(';','').replace('--','').replace('.','').replace(',','').replace('(','').replace(')','').replace("'",'').replace('"','').replace(':','').replace('“','').split()
    return [w for w in x if w not in stop_words]

In [ ]:
cleaned_df['overview_cleaned'] = cleaned_df['overview'].apply(lambda x: overview_cleaning(x))

#### Unikke ord
Her oprettes variablen der indheholder alle unikke ord og vil blive anvendt flere gange senere.

In [ ]:
bigrams = []
for sentence in cleaned_df['overview_cleaned']:
    bigrams += ([w + ' ' + sentence[i+1] for i, w in enumerate(sentence) if i < len(sentence)-1])

bigrams = list(set(bigrams))

In [ ]:
unigrams = []
for token in cleaned_df['overview_cleaned']:
    unigrams.extend(token)
unigrams = ([word for word in unigrams])
unigrams = list(set(unigrams))

### TF
Her beregnes Term Frequency, funktionen tf_calculator returnerer en dictionary med de ord der er anvendt for en vilkårlig film som key og antal gange ét ord indgår i beskrivelsen for den vilkårlige film og dette divideres med det totale antal af ord i beskrivelsen og dette er vores value for ordet.

In [ ]:
def tf_calculator(x, is_unigram):
    bow_dict = {}
    if is_unigram:
        bow_dict = dict.fromkeys(x, 0)
        for w in x:
            bow_dict[w] += 1
        for key in bow_dict:
            bow_dict[key] = bow_dict[key] / len(x)
    else:
        bow_dict = dict.fromkeys([w + ' ' + x[i+1] for i, w in enumerate(x) if i < len(x)-1], 0)
        for i in range(len(x)-1):
            bow_dict[x[i] + ' ' + x[i+1]] += 1
        for key in bow_dict:
            bow_dict[key] = bow_dict[key] / x.count(key.split()[0])
    return(bow_dict)

In [ ]:
tf_bigrams = cleaned_df['overview_cleaned'].apply(lambda x: tf_calculator(x, is_unigram=False))
tf_unigrams = cleaned_df['overview_cleaned'].apply(lambda x: tf_calculator(x, is_unigram=True))

### IDF
Her beregnes Inverse Document Frequency. funktionen idf_calculator returnerer en dictionary med unikke ord som keys og som value er det antal af film divideret med antallet af gange ordet x forekommer - Hvis ordet x forekommer i alle film vil værdien være 4803, da vi har i alt 4803 film og dette er også tilfældet selvom ordet x forekommer mere end 1 gang i flere film, da dette ikke tages højde for.

# Bør kun eksekveres 1 gang, hvis "data/idf_dict.pkl" eller "data/idf_dict.pkl_unigram" ikke eksisterer

### N antal gange unigram optræder i film

In [ ]:
#idf_dict = dict.fromkeys(unigrams, 0)

In [ ]:
#temp_idf_dict = {k: v for k, v in idf_dict.items() if v < 1}
#print(len(temp_idf_dict))

In [ ]:
#for word in temp_idf_dict:
#    is_word_existing = True
#    for movie in cleaned_df['overview_cleaned']:
#        movie = ([w for w in movie])
#        if word in movie:
#            idf_dict[word] += 1
#            is_word_existing = False
#    if is_word_existing:
#        print(word)

In [ ]:
#with open("data/idf_dict_unigram.pkl", "wb") as file:
#    pickle.dump(idf_dict, file)

### N antal gange bigram optræder i film

In [ ]:
#idf_dict = dict.fromkeys(bigrams, 0)

In [ ]:
#temp_idf_dict = {k: v for k, v in idf_dict.items() if v < 1}
#print(len(temp_idf_dict))

In [ ]:
#for word in temp_idf_dict:
#    is_word_existing = True
#    for movie in cleaned_df['overview_cleaned']:
#        movie = ([w + ' ' + movie[i+1] for i, w in enumerate(movie) if i < len(movie)-1])
#        if word in movie:
#            idf_dict[word] += 1
#            is_word_existing = False
#    if is_word_existing:
#        print(word)

In [ ]:
#with open("data/idf_dict.pkl", "wb") as file:
#    pickle.dump(idf_dict, file)

******************************************************************************************************************

******************************************************************************************************************

In [ ]:
with open("data/idf_dict.pkl", "rb") as file:
    bigrams_dict = pickle.load(file)

In [ ]:
with open("data/idf_dict_unigram.pkl", "rb") as file:
    unigrams_dict = pickle.load(file)

In [ ]:
print(sorted(list(bigrams_dict.items()), key=lambda x: x[1], reverse=True)[:50])

In [ ]:
print(sorted(list(unigrams_dict.items()), key=lambda x: x[1], reverse=True)[:50])

In [ ]:
def idf_calculator(ngram_dict):
    for key in ngram_dict:
        ngram_dict[key] = math.log10(len(cleaned_df)/ngram_dict[key])
    return ngram_dict

In [ ]:
idf_bigrams = idf_calculator(bigrams_dict)

In [ ]:
idf_unigrams = idf_calculator(unigrams_dict)

### TF-IDF
funktionen returnerer en liste af dictionaries, hvor værdierne i disse dictionaries ganges sammen med den tilsvarende idf værdi - altså idf værdien for ordet "peter" ganges med tf værdien for samme ord, altså "peter".

In [ ]:
def tfidf_calculator(tf, idf):
    for key in tf:
        if key in idf:
            tf[key] = tf[key] * idf[key]
    return tf

In [ ]:
tfidf_bigrams = tf_bigrams.apply(lambda x: tfidf_calculator(x, idf_bigrams))
tfidf_unigrams = tf_unigrams.apply(lambda x: tfidf_calculator(x, idf_unigrams))

## Kombinering af unigrams og bigrams

In [ ]:
def bigram_calculator():
    for i in range(len(cleaned_df)):
        for uni in tfidf_unigrams[i]:
            for bi in tfidf_bigrams[i]:
                if uni == bi.split()[0]:
                    tfidf_bigrams[i][bi] = tfidf_bigrams[i][bi] * tfidf_unigrams[i][uni]
    return tfidf_bigrams

In [ ]:
tfidf_bigrams = bigram_calculator()

### Matricen
Funktionen tfidf_matrix returnerer en matrice, der går igennem alle unikke ord og giver tilføjer værdien 0, hvis ordet ikke eksisterer i, beskrivelsen af en given film ellers tilføjes TF-IDF værdien, hvis ordet eksisterer i beskrivelsen til filmen.  

In [ ]:
def tfidf_matrix(tfidf_dict, ngrams):
    return {i: tfidf_dict[ngram] for i, ngram in enumerate(ngrams) if ngram in tfidf_dict}

In [ ]:
#cleaned_df['bigram_matrix'] = tfidf_bigrams.apply(lambda x: tfidf_matrix(x, bigrams))
#cleaned_df['unigram_matrix'] = tfidf_unigrams.apply(lambda x: tfidf_matrix(x, unigrams))

In [ ]:
#cleaned_df.to_pickle("data/tfidf_dataframe.pkl")

# Nedenstående kode kan køres seperat (OBS kør imports først)

In [2]:
cleaned_df = pd.read_pickle("data/tfidf_dataframe.pkl")

## Cosine Simularity
Der er umiddelbart to metoder vi kan anvende, da vi har lister af værdier, den metode, hvor numpy anvendes, siges at være den "hurtigeste" i form af runtime.

In [3]:
def cosine_similarity(vector_of_interest, comparison_vector):
    cos_sim = 0
    for key in vector_of_interest:
        if key in comparison_vector:
            cos_sim += (vector_of_interest[key] * comparison_vector[key])
    length_voi = [vector_of_interest[i]**2 for i in vector_of_interest]
    length_cv = [comparison_vector[i]**2 for i in comparison_vector]
    
    return cos_sim / (math.sqrt(sum(length_voi)) * math.sqrt(sum(length_cv)))

In [4]:
def get_recommendations(title, matrix):
    main_movie_vector = cleaned_df.loc[cleaned_df['title'] ==  title][matrix].values[0]
    cos_sim = [cosine_similarity(main_movie_vector, cleaned_df[matrix][i]) for i in range(len(cleaned_df) -1)]
    
    return [cleaned_df.iloc[i]['title'] for i in pd.Series(cos_sim).nlargest(11).index[1:]]

In [5]:
top_movies = get_recommendations('Star Wars', 'bigram_matrix')

In [6]:
print(*top_movies, sep="\n")

The Empire Strikes Back
Return of the Jedi
Roadside
The Lost Boys
Star Wars: Episode III - Revenge of the Sith
Gangs of New York
The Last Dragon
Haywire
The Legend of Hercules
Alpha and Omega


In [7]:
top_movies = get_recommendations('Star Wars', 'unigram_matrix')

In [8]:
print(*top_movies, sep="\n")

The Empire Strikes Back
Return of the Jedi
Shanghai Noon
Arbitrage
The Princess Diaries 2: Royal Engagement
Prince of Persia: The Sands of Time
History of the World: Part I
The Princess Bride
Star Wars: Episode III - Revenge of the Sith
Mirror Mirror
